# AI - CA1 - Searching Algorithms
## Introduction
In this assignment, we are going to use three different search algorithms.
The first two algorithms are uninformed search algorithms, BFS and IDS. The last algorithm is the A* algorithm which is an informed search algorithm and we implement this algorithm with two different heuristics in this assignment.
The algorithms will be thoroughly examined in the next parts



## Part I

### Modeling the Problem

#### The States
In this problem, the states are Map objects. A Map object contains a list of patient positions, a list of obstacle positions, a dictionary with the hospital positions as keys and their respective remaining capacities as values and the position of the ambulance on the map. There is also an additional state named evaluation which will be used for comparing two states in the A* algorithm, this field was needed for the Priority Queue data structure to work. 

#### Initial State
The initial state contains the positions of the obstacles, patients and the ambulance and the position and the capacity of the hospitals on the starting map which is given in a file. The file is loaded and the position of the patients, obstacles and the ambulance is saved along with the position and capacity of the hospitals.

#### Goal State
The goal state is when no patients are remaining in the map.

#### Actions
In each state, the ambulance itself can move up, down, left or right, thus building up to 4 new states while being expanded, and these new states are added to the frontier set of the problem.

#### Transition Model
If the ambulance goes from cell A to cell B, it ends up in cell B; and if there is already a patient in cell B, the patient will be moved one cell in the direction of the ambulance's movement. If a patient is moved into a hospital with a capacity greater than zero, the patient is removed from the following states.

#### Path Cost
The number of moves the ambulance has made to transfer all of the patients to the hospitals.

In [1]:
from copy import deepcopy

OBSTACLE_CHAR = '#'
PATIENT_CHAR = 'P'
AMBULANCE_CHAR = 'A'
HOSPITAL_CHAR = ('0', '1', '2', '3')
class Map:
    def __init__(
        self, 
        empty: bool = False, 
        obstacles: list = None, 
        patients: list = None, 
        ambulance: list = None, 
        hospitals: dict = None,
        evaluation: int = None,
        cost: int = 0,
    ):
        if empty:
            self.hospitals = {}
            self.ambulance = None
            self.obstacles = []
            self.patients = []
            self.cost = 0
            self.evaluation = None
        else:
            self.hospitals = hospitals
            self.ambulance = ambulance
            self.obstacles = obstacles
            self.patients = patients
            self.cost = cost
            self.evaluation = evaluation
    
    def __eq__(
        self,
        other
    ):
        if (
            self.patients == other.patients
            and self.hospitals == other.hospitals
            and self.ambulance == other.ambulance
        ):
            return True
        return False

    def __hash__(
        self
    ):
        return hash(str(
            [
             self.ambulance,
             self.patients,
             self.hospitals
            ]
        ))
    
    def __gt__(
        self,
        other
    ):
        if self.evaluation > other.evaluation:
            return True
        return False

    def __ge__(
        self,
        other
    ):
        if self.evaluation >= other.evaluation:
            return True
        return False

    def set_evaluation(
        self,
        evaluation
    ):
        self.evaluation = evaluation

    def read_map_from_file(
        self, 
        file_address: str
    ) -> None:
        with open(file_address, 'r') as map_file:
            x = -1
            for line in map_file:
                x += 1
                y = -1
                for char in line:
                    y += 1
                    if char == OBSTACLE_CHAR:
                        self.obstacles.append((x,y))
                    elif char == PATIENT_CHAR:
                        self.patients.append([x,y])
                    elif char == AMBULANCE_CHAR:
                        self.ambulance = [x,y]
                    elif char in HOSPITAL_CHAR:
                        self.hospitals[(x,y)] = int(char)

    def build_new_states(
        self
    ) -> list:
        new_states = []
        diff = [
            [-1,0], 
            [1,0], 
            [0,-1], 
            [0,1]
        ]
        for dxy in diff:
            new_xy = [
                self.ambulance[0] + dxy[0], 
                self.ambulance[1] + dxy[1]
            ]
            if new_xy in self.patients:
                newer_xy = [new_xy[0] + dxy[0], new_xy[1] + dxy[1]]
                if tuple(newer_xy) in self.hospitals.keys() and self.hospitals[tuple(newer_xy)] > 0:
                    new_patients = deepcopy(self.patients)
                    new_patients.remove(new_xy)
                    new_hospitals = deepcopy(self.hospitals)
                    new_hospitals[tuple(newer_xy)] -= 1
                    new_state = Map(
                        patients=new_patients,
                        hospitals=new_hospitals,
                        obstacles=self.obstacles,
                        ambulance=deepcopy(new_xy),
                        cost = self.cost + 1
                    )
                    new_states.append(new_state)
                elif newer_xy not in self.patients and tuple(newer_xy) not in self.obstacles:
                    new_patients = deepcopy(self.patients)
                    for idx, patient in enumerate(new_patients):
                        if patient == new_xy:
                            new_patients[idx][0] += dxy[0]
                            new_patients[idx][1] += dxy[1]                                
                    new_state = Map(
                        patients=new_patients,
                        hospitals=deepcopy(self.hospitals),
                        obstacles=self.obstacles,
                        ambulance=deepcopy(new_xy),
                        cost = self.cost + 1
                    )
                    new_states.append(new_state)
            elif tuple(new_xy) not in self.obstacles:
                new_state = Map(
                    patients=deepcopy(self.patients),
                    hospitals=deepcopy(self.hospitals),
                    obstacles=self.obstacles,
                    ambulance=deepcopy(new_xy),
                    cost = self.cost + 1
                )
                new_states.append(new_state)
        return new_states
    
    def is_goal_state(
        self
    ) -> bool:
        return False if len(self.patients) else True

### Run Tests Utility Function

The following function gets the search function, repeat count (to get the average running time of the algorithm), and the test file names as arguments and runs the search function on the given tests and returns the result. (A heuristic function can be given if needed.)

In [2]:
def run_tests(
    search_function,
    repeat_count: int,
    test_filenames: list,
    heuristic_func=None
) -> dict:
    final_result = {}
    for j in range(len(test_filenames)):
        running_time_average = 0
        search_func_args = [
            test_filenames[j],
        ]
        if heuristic_func:
            search_func_args.append(heuristic_func)
        for i in range(repeat_count):
            partial_result = search_function(*search_func_args)
            running_time_average += partial_result['Running time']
        running_time_average /= repeat_count
        result = {
            'Test No.': j + 1,
            'States explored': partial_result['States explored'],
            'Unique states explored': partial_result['Unique states explored'],
            'Steps to reach the goal state': partial_result['Steps to reach the goal state'],
            'Running time': running_time_average
        }
        final_result[j + 1] = result
    return final_result

## Part II

### BFS Algorithm

The BFS algorithm expands the shallowest node in the frontier set in each iteration, it is a complete solution and it's also optimal if the step costs are all equal as it yields the path with the least number of steps.
BFS uses a FIFO queue as a data structure for it's frontier set as this results in the shallowest node being popped from the queue each time.

Time Complexity: $O(b^d)$<br>
Space Complexity: $O(b^d)$<br>
where $b$ is the maximum branching factor of the search tree and $d$ is the maximum depth of the optimal solution.

The biggest problem of the BFS algorithm is its space inefficiency.

In [3]:
def bfs(
    file_path
):
    from queue import Queue
    import time

    initial_state = Map(empty=True)
    initial_state.read_map_from_file(file_address=file_path)
    explored_set = set()
    frontier = Queue()
    frontier.put(initial_state)

    states_explored = 0
    starting_time = time.time()
    while not frontier.empty():
        states_explored += 1
        current_state = frontier.get()
        explored_set.add(current_state)
        if current_state.is_goal_state():
            ending_time = time.time()
            return {
                'States explored': states_explored,
                'Unique states explored': len(explored_set),
                'Steps to reach the goal state': current_state.cost,    
                'Running time': ending_time - starting_time,
            }
        next_states = current_state.build_new_states()
        for i in next_states:
            if i not in explored_set:
                frontier.put(i)
    print('no answer :(')
    return

### Testing the BFS algorithm

In the following snippet, the BFS algorithm is tested 3 times on each test case and the average is calculated for each field.

The results are shown in a table below.

In [4]:
test_files = [
    './testcases/test1.txt',
    './testcases/test2.txt',
    './testcases/test3.txt'
]

bfs_results = run_tests(
    bfs,
    3,
    test_files
)

In [5]:
from prettytable import PrettyTable

bfs_table = PrettyTable()
bfs_table.field_names = list(bfs_results[1].keys())
bfs_table.add_row(list(bfs_results[1].values()))
bfs_table.add_row(list(bfs_results[2].values()))
bfs_table.add_row(list(bfs_results[3].values()))
print('BFS Results Table:')
print(bfs_table)

BFS Results Table:
+----------+-----------------+------------------------+-------------------------------+--------------------+
| Test No. | States explored | Unique states explored | Steps to reach the goal state |    Running time    |
+----------+-----------------+------------------------+-------------------------------+--------------------+
|    1     |       720       |          396           |               11              | 0.0517731507619222 |
|    2     |      37838      |         15051          |               27              | 2.5387157599131265 |
|    3     |      306320     |         31876          |               39              | 17.84168267250061  |
+----------+-----------------+------------------------+-------------------------------+--------------------+


## Part III


### IDS Algorithm

> In order to understand the IDS algorithm, we should first understand the DFS algorithm. DFS algorithm always expands the deepest node in the frontier set, thus going as deep as possible in a branch before traversing back up. It's slower than the BFS algorithm and doesn't always yield an optimal solution, in fact it yields the first solution it finds. But an advantage of the DFS algorithm is its linear space complexity which is $O(bm)$ where $m$ is the max depth of any possible path in the state space.

The IDS algorithm combines the space efficiency of the DFS algorithm with the speed of the BFS algorithm by running DFS algorithms only within a limited depth in the tree. In this algorithm the shallow nodes are visited multiple times but as the nodes get deeper, the less they get visited (The last level nodes are only visited once, the second last level nodes are only visited twice and so on.) but since most of the nodes are in the bottom levels of a tree, this algorithm works pretty well. As this algorithm runs a DFS in each iteration, the data structure used for storing the frontier set is the LIFO queue data structure, as the deepest node is always returned in this data structure, however if the node is of depth $k$ ($k$ being the max depth of the corresponding iteration.) it won't be expanded anymore.

Time Complexity: $O(b^d)$<br>
Space Complexity: $O(bd)$<br>
As seen here, the space complexity of this algorithm is of linear order, which is a great advantage over the BFS algorithm.

In [6]:
def ids(
    file_path
):
    import time

    def search(max_depth: int):
        from queue import LifoQueue

        initial_state = Map(empty=True)
        initial_state.read_map_from_file(file_address=file_path)
        explored_set = {}
        frontier = []
        frontier.append(initial_state)

        states_explored = 0
        while len(frontier):
            states_explored += 1
            current_state = frontier.pop()
            explored_set[current_state.__hash__] = current_state.cost
            if current_state.is_goal_state():
                return {
                    'States explored': states_explored,
                    'Unique states explored': len(explored_set.keys()),
                    'Steps to reach the goal state': current_state.cost,    
                }
            if current_state.cost < max_depth:
                next_states = current_state.build_new_states()
                for i in next_states:
                    if (i.__hash__ not in explored_set 
                    or explored_set[i.__hash__] > i.cost):
                        frontier.append(i)
        return None


    max_depth = 0
    result = None
    starting_time = time.time()
    while not result:
        result = search(max_depth)
        max_depth += 1
    ending_time = time.time()
    result['Running time'] = ending_time - starting_time
    return result

### Testing the IDS algorithm

In the following snippet, the IDS algorithm is tested 3 times on each test case and the average is calculated for each field.

The results are shown in a table below.

In [7]:
ids_results = run_tests(
    ids,
    3,
    test_files      
)

In [8]:
from prettytable import PrettyTable

ids_table = PrettyTable()
ids_table.field_names = list(ids_results[1].keys())
ids_table.add_row(list(ids_results[1].values()))
ids_table.add_row(list(ids_results[2].values()))
ids_table.add_row(list(ids_results[3].values()))
print('IDS Results Table:')
print(ids_table)

IDS Results Table:
+----------+-----------------+------------------------+-------------------------------+---------------------+
| Test No. | States explored | Unique states explored | Steps to reach the goal state |     Running time    |
+----------+-----------------+------------------------+-------------------------------+---------------------+
|    1     |       579       |          406           |               11              | 0.07761247952779134 |
|    2     |      48862      |         16147          |               27              |  14.185588916142782 |
|    3     |      142647     |         34072          |               39              |  46.732839504877724 |
+----------+-----------------+------------------------+-------------------------------+---------------------+


## Part IV

### A* Algorithm

The A\* algorithm (unlike the previous ones) is an informed search algorithm, which means it has some hints and knows which states might lead to the goal state faster. In general, the informed search algorithms have a heuristic function which predicts the cost needed to go from each state to the goal state. However, this may result in suboptimal paths because the algorithm has no idea of the cost already paid to reach that state. The A\* algorithm solves this problem by using an evaluation function which estimates the total cost of reaching the goal state from each state (as opposed to the heuristic function which only estimates the cost of reaching the goal state starting from each state). The evaluation function sums the estimated cost from the heuristic function for each state with the actual cost of reaching the state and uses this value as the estimated total cost of reaching the goal state. The evaluation function is calculated as seen below:
$$f(n) = g(n) + h(n)$$
> Where $f(n)$ is the evaluation function, $g(n)$ is the actual cost of reaching the $n_{th}$ state and $h(n)$ is the heuristic function

 The data structure used to store the frontier set is a Priority Queue which uses the result of the evaluation function for each state as that state's priority and returns the state with the highest priority (lowest evaluation function value which is the lowest total estimated cost to reach the goal state).

 The A\* algorithm is executed with two different heuristics in this assignment, which are thoroughly explained below.

In [9]:
def a_star(
    file_path,
    heuristic
):
    from queue import PriorityQueue
    import time

    initial_state = Map(empty=True)
    initial_state.read_map_from_file(file_address=file_path)
    explored_set = set()
    frontier = PriorityQueue()
    h_initial = heuristic(initial_state)
    initial_state.set_evaluation(h_initial)
    frontier.put((h_initial, initial_state))
    states_explored = 0
    starting_time = time.time()
    while not frontier.empty():
        states_explored += 1
        current_state = frontier.get()[1]
        explored_set.add(current_state)
        if current_state.is_goal_state():
            ending_time = time.time()
            return {
                'States explored': states_explored,
                'Unique states explored': len(explored_set),
                'Steps to reach the goal state': current_state.cost,    
                'Running time': ending_time - starting_time,
            }
        next_states = current_state.build_new_states()
        for i in next_states:
            if i not in explored_set:
                h = heuristic(i)
                f = i.cost + h
                i.set_evaluation(f)
                frontier.put((f, i))
    print('no answer :(')
    return

### Heuristic Function No.1

#### The Sum of Manhattan Distance Between Each Patient and the Nearest Hospital Which is not Full

In this part, the heuristic used is the manhattan distance between each patient and the nearest hospital which is not full already. This heuristic works because it's value gets lower as the patients get closer to the hospitals, which is what we need (all of the patients should finally get into a hospital which has a vacant room for them).
This heuristic is admissible because all of the patients should at least make this number of moves to get to a hospital, so it will never overestimate the cost.

In [10]:
def patient_hospital_manhattan_distance_heuristic(
    state: Map
) -> int:
    total = 0
    for patient in state.patients:
        minimum_distance = float('inf')
        for hospital in state.hospitals.keys():
            if not state.hospitals[hospital]:
                continue
            distance = abs(patient[0] - hospital[0]) + abs(patient[1] - hospital[1])
            if distance < minimum_distance:
                minimum_distance = distance
        total += minimum_distance
    return total

In [11]:
a_star_1_results = run_tests(
    a_star,
    3,
    test_files,
    patient_hospital_manhattan_distance_heuristic
)

In [12]:
from prettytable import PrettyTable

a_star_table = PrettyTable()
a_star_table.field_names = list(a_star_1_results[1].keys())
a_star_table.add_row(list(a_star_1_results[1].values()))
a_star_table.add_row(list(a_star_1_results[2].values()))
a_star_table.add_row(list(a_star_1_results[3].values()))
print('A* Results Table\n(Heuristic: Sum of manhattan distances between each patient and the nearest non-full hospital):')
print(a_star_table)

A* Results Table
(Heuristic: Sum of manhattan distances between each patient and the nearest non-full hospital):
+----------+-----------------+------------------------+-------------------------------+----------------------+
| Test No. | States explored | Unique states explored | Steps to reach the goal state |     Running time     |
+----------+-----------------+------------------------+-------------------------------+----------------------+
|    1     |       181       |          140           |               11              | 0.011588652928670248 |
|    2     |       8350      |          5495          |               27              |  0.6703280607859293  |
|    3     |      23373      |         10719          |               39              |  1.5249589284261067  |
+----------+-----------------+------------------------+-------------------------------+----------------------+


### Weighted A* Algorithm

Another way to make the A* algorithm faster, is to multiply the heuristic function's result by a number ($\alpha$). This may result in the found path being suboptimal, but can make the algorithm many times faster. The cost from the weighted A* algorithm will be at most $\alpha$ times the cost which the standard A* algorithm gives.

In [13]:
def weighted_patient_hospital_manhattan_distance_heuristic(
    state: Map
) -> int:
    return 2 * patient_hospital_manhattan_distance_heuristic(state)

In [14]:
weighted_a_star_results = run_tests(
    a_star,
    3,
    test_files,
    weighted_patient_hospital_manhattan_distance_heuristic
)

In [15]:
from prettytable import PrettyTable

a_star_table = PrettyTable()
a_star_table.field_names = list(weighted_a_star_results[1].keys())
a_star_table.add_row(list(weighted_a_star_results[1].values()))
a_star_table.add_row(list(weighted_a_star_results[2].values()))
a_star_table.add_row(list(weighted_a_star_results[3].values()))
print('Weighted A* Results Table:\n(The same heuristic as above, only multiplied by 2)')
print(a_star_table)

Weighted A* Results Table:
(The same heuristic as above, only multiplied by 2)
+----------+-----------------+------------------------+-------------------------------+----------------------+
| Test No. | States explored | Unique states explored | Steps to reach the goal state |     Running time     |
+----------+-----------------+------------------------+-------------------------------+----------------------+
|    1     |        39       |           36           |               11              | 0.002949237823486328 |
|    2     |       1272      |          1002          |               27              | 0.09748999277750652  |
|    3     |       9169      |          4806          |               39              |  0.6049313545227051  |
+----------+-----------------+------------------------+-------------------------------+----------------------+


### Heuristic Function No.2
#### The Sum of Manhattan Distances Between each Patient and the Ambulance

In this part, the heuristic used is the manhattan distance between each patient and the ambulance. This heuristic works because it's value gets lower as the ambulance get closer to the patients, and then it can move them to the hospitals.
This heuristic is admissible because the ambulance should reach each patient inevitably and the heuristic value is the minimum steps needed to reach each patient, so it will always be lower than the actual cost.
As seen in the following results, this heuristic works better than the first one because we want to optimize the number of steps taken by the ambulance, which is done when searching with this heuristic while the first heuristic was more focused on optimizing the count of steps taken by the patients rather than the ambulance.

In [16]:
def ambulance_patient_manhattan_distance(
    state: Map
) -> int:
    total = 0
    for patient in state.patients:
        total += abs(patient[0] - state.ambulance[0]) + abs(patient[1] - state.ambulance[1])
    return total

In [17]:
a_star_2_results = run_tests(
    a_star,
    3,
    test_files,
    ambulance_patient_manhattan_distance
)

In [18]:
from prettytable import PrettyTable

a_star_table = PrettyTable()
a_star_table.field_names = list(a_star_2_results[1].keys())
a_star_table.add_row(list(a_star_2_results[1].values()))
a_star_table.add_row(list(a_star_2_results[2].values()))
a_star_table.add_row(list(a_star_2_results[3].values()))
print('A* Results Table:')
print(a_star_table)

A* Results Table:
+----------+-----------------+------------------------+-------------------------------+----------------------+
| Test No. | States explored | Unique states explored | Steps to reach the goal state |     Running time     |
+----------+-----------------+------------------------+-------------------------------+----------------------+
|    1     |       132       |          122           |               11              | 0.008227348327636719 |
|    2     |       8250      |          5889          |               27              |  0.6537110805511475  |
|    3     |      11584      |          8098          |               39              |  0.7929476102193197  |
+----------+-----------------+------------------------+-------------------------------+----------------------+


## Part V

### Conclusion and Comparison

In [19]:
from prettytable import PrettyTable

algorithm_results = [
    bfs_results, 
    ids_results, 
    a_star_1_results, 
    a_star_2_results
]
algorithm_names = [
    'BFS',
    'IDS',
    'A*',
    'A*'
]

for idx, res in enumerate(algorithm_results):
    for v in res.values():
        try:
            del v['Test No.']
        except:
            pass
        v['Algorithm'] = algorithm_names[idx]


test_1_table = PrettyTable()
test_1_table.field_names = list(reversed(list(bfs_results[1].keys())))
test_1_table.add_row(list(reversed(list(bfs_results[1].values()))))
test_1_table.add_row(list(reversed(list(ids_results[1].values()))))
test_1_table.add_row(list(reversed(list(a_star_1_results[1].values()))))
test_1_table.add_row(list(reversed(list(a_star_2_results[1].values()))))
print('Test 1 Results Table:')
print(test_1_table)

test_2_table = PrettyTable()
test_2_table.field_names = list(reversed(list(bfs_results[2].keys())))
test_2_table.add_row(list(reversed(list(bfs_results[2].values()))))
test_2_table.add_row(list(reversed(list(ids_results[2].values()))))
test_2_table.add_row(list(reversed(list(a_star_1_results[2].values()))))
test_2_table.add_row(list(reversed(list(a_star_2_results[2].values()))))
print('Test 2 Results Table:')
print(test_2_table)

test_3_table = PrettyTable()
test_3_table.field_names = list(reversed(list(bfs_results[3].keys())))
test_3_table.add_row(list(reversed(list(bfs_results[3].values()))))
test_3_table.add_row(list(reversed(list(ids_results[3].values()))))
test_3_table.add_row(list(reversed(list(a_star_1_results[3].values()))))
test_3_table.add_row(list(reversed(list(a_star_2_results[3].values()))))
print('Test 3 Results Table:')
print(test_3_table)

Test 1 Results Table:
+-----------+----------------------+-------------------------------+------------------------+-----------------+
| Algorithm |     Running time     | Steps to reach the goal state | Unique states explored | States explored |
+-----------+----------------------+-------------------------------+------------------------+-----------------+
|    BFS    |  0.0517731507619222  |               11              |          396           |       720       |
|    IDS    | 0.07761247952779134  |               11              |          406           |       579       |
|     A*    | 0.011588652928670248 |               11              |          140           |       181       |
|     A*    | 0.008227348327636719 |               11              |          122           |       132       |
+-----------+----------------------+-------------------------------+------------------------+-----------------+
Test 2 Results Table:
+-----------+--------------------+--------------------------